Inspired by Deep Learning for Computer Vision with Python [Rosebrock]  
Chapter 14  
LeNet: MNIST

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# load the MNIST dataset of 70,000 hand-written digits
# 60,000 for training and 10,000 for testing
((trainX, trainY), (testX, testY)) = tf.keras.datasets.mnist.load_data()

# reshape the design matrix to: num_samples x rows x columns x depth
trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX = testX.reshape((testX.shape[0], 28, 28, 1))

# scale data to the range of [0, 1]
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
# build the model
height = 28
width = 28
depth = 1
classes = 10

model = tf.keras.models.Sequential()
inputShape = (height, width, depth)

# first set of CONV => RELU => POOL layers
model.add(tf.keras.layers.Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# second set of CONV => RELU => POOL layers
model.add(tf.keras.layers.Conv2D(50, (5, 5), padding="same"))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# first (and only) set of FC => RELU layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(500))
model.add(tf.keras.layers.Activation("relu"))

# softmax classifier
model.add(tf.keras.layers.Dense(classes))
model.add(tf.keras.layers.Activation("softmax"))
model.summary()

# initialize the gradient descent optimizer
sgd = tf.keras.optimizers.SGD(0.01)

# compile the model
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [ ]:
# train the model
EPOCHS = 20
H = model.fit(x=trainX, y=trainY, validation_data=(testX, testY), epochs=EPOCHS, batch_size=128, verbose=1)

In [ ]:
# evaluate the network
predictions = model.predict(testX, batch_size=128)
label_names = [str(x) for x in lb.classes_]
report = classification_report(
    y_true=testY.argmax(axis=1),
    y_pred=predictions.argmax(axis=1),
    target_names=label_names)
print(report)

# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, EPOCHS), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, EPOCHS), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, EPOCHS), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, EPOCHS), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()